# Data pre-processing  

> [https://github.com/BMClab/covid19](https://github.com/BMClab/covid19)  
> [Laboratory of Biomechanics and Motor Control](http://pesquisa.ufabc.edu.br/bmclab/)  
> Federal University of ABC, Brazil

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Environment</a></span></li><li><span><a href="#Helping-functions" data-toc-modified-id="Helping-functions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Helping functions</a></span></li></ul></li><li><span><a href="#Load-dataset" data-toc-modified-id="Load-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load dataset</a></span><ul class="toc-item"><li><span><a href="#Checking-for-missing-values" data-toc-modified-id="Checking-for-missing-values-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Checking for missing values</a></span></li><li><span><a href="#Data-types-and-memory-usage" data-toc-modified-id="Data-types-and-memory-usage-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data types and memory usage</a></span></li></ul></li><li><span><a href="#Pre-processing" data-toc-modified-id="Pre-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pre-processing</a></span><ul class="toc-item"><li><span><a href="#Select-athletes-who-ran-at-least-once-in-2019" data-toc-modified-id="Select-athletes-who-ran-at-least-once-in-2019-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Select athletes who ran at least once in 2019</a></span></li><li><span><a href="#Save-the-current-dataframe-as-the-default-raw-dataset" data-toc-modified-id="Save-the-current-dataframe-as-the-default-raw-dataset-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Save the current dataframe as the default raw dataset</a></span></li><li><span><a href="#Select-'run'-as-activity-type" data-toc-modified-id="Select-'run'-as-activity-type-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Select 'run' as activity type</a></span></li><li><span><a href="#Transform-units-to-kilometers-and-minutes" data-toc-modified-id="Transform-units-to-kilometers-and-minutes-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Transform units to kilometers and minutes</a></span></li><li><span><a href="#Update-age-groups-to-2019" data-toc-modified-id="Update-age-groups-to-2019-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Update age groups to 2019</a></span></li><li><span><a href="#Merge-the-9-age-groups-into-3-groups" data-toc-modified-id="Merge-the-9-age-groups-into-3-groups-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Merge the 9 age groups into 3 groups</a></span></li><li><span><a href="#Correct-some-country-names" data-toc-modified-id="Correct-some-country-names-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Correct some country names</a></span></li></ul></li><li><span><a href="#Export-dataset" data-toc-modified-id="Export-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export dataset</a></span><ul class="toc-item"><li><span><a href="#Full-dataset-with-running-activities" data-toc-modified-id="Full-dataset-with-running-activities-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Full dataset with running activities</a></span></li><li><span><a href="#Test-file" data-toc-modified-id="Test-file-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Test file</a></span></li></ul></li></ul></div>

## Setup

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
%load_ext watermark  

%watermark
%watermark --iversions

### Environment

In [ ]:
path2 = r'./../data/'

pd.set_option('display.float_format', lambda x: '%.4g' % x)

### Helping functions

In [ ]:
df = pd.read_csv(os.path.join(path2, 'run_ww_2019.csv'), sep='\t', header=0, nrows=1)
columns = df.columns
columns

In [ ]:
def read(fname):
    """Read dataset file as pandas dataframe."""
    y = pd.read_csv(fname, sep='\t', header=0, parse_dates=['datetime'], verbose=False,
                    usecols=['athlete', 'gender', 'age_group', 'datetime', 'activity',
                             'distance', 'duration', 'country', 'major'],
                    dtype={'athlete': 'category', 'gender': 'category',
                           'age_group': 'category', 'activity': 'category',
                           'distance': 'int32', 'duration': 'int32',
                           'country': 'category', 'major': 'category'})
    return y

## Load dataset

Let's load the dataset as a [pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/index.html), this will simplify the data handling.

We will load the entire dataset in RAM memory. This will work for the present dataset in a computer with at least 8 GB of RAM memory and it's not scalable (unless you keep buying more RAM memory). For a scalable solution to handle a bigger dataset see for example [Dask](https://dask.org/) or [datatable](https://github.com/h2oai/datatable).  

Let's create a function to read the dataset defining some features as categorical variables, which will decrease memory use once the data is loaded.

In [ ]:
years = ['2019', '2020']
df = pd.DataFrame()
for year in tqdm(years):
    df = df.append(read(os.path.join(path2, 'run_ww_' + year + '.csv')), ignore_index=True)
    if year == years[-1]:
        cols = df.select_dtypes(include='O').columns.to_list()
        df[cols] = df[cols].astype('category')
        df.sort_values(['datetime', 'athlete'], inplace=True)
df

### Checking for missing values

In [ ]:
n = 0
for col in df:
    null = df[df[col].isnull()]['athlete'].unique().to_list()
    if null and col != 'challenge' and col != 'country':
        print('Athlete: {}, null value in {}'.format(null, col))
        n = 1
if n == 0:
    print('No missing values found (except in columns challenge and country).')

### Data types and memory usage

In [ ]:
df.info(memory_usage='deep')

There are about 150 millions registers in the loaded dataset (14.5 million rows times 10 columns). If we had loaded the data as `float64` or `string` Python objects, we would amount to about 5 GB of memory usage, more than 10 times what actually is being used as a pandas dataframe with the defined object types.

In [ ]:
ndays = df['datetime'].dt.date.value_counts().size
print('Number of days:', ndays)
nathletes = df['athlete'].unique().size
print('Number of athletes:', nathletes)
nactivities = df['activity'].size
print('Number of activities:', nactivities)
nactivities2 = df['activity'].unique().size
print('Number of types of activities:', nactivities2)
duration = df['duration'].sum()/ndays/nathletes
print('Average duration of activity per day per athlete: {:.0f} s'.format(duration))

## Pre-processing

If there are registers of running with zero distance or duration of run, remove them.

In [ ]:
zeroduration = df[(df['activity']=='run') & 
                  ((df['distance'] == 0) | (df['duration'] == 0))]
print('Number of run activities with zero distance or duration:', zeroduration.shape[0])
if zeroduration.size:
    df = df.drop(index=zeroduration.index)
    df['athlete'] = df['athlete'].cat.remove_unused_categories()

### Select athletes who ran at least once in 2019

To investigate a COVID-19 related effect comparing 2019 with 2020, an athlete should have ran at least once in 2019.  
Let's find out if there are athletes in 2020 who are not in 2019 and remove them.

In [ ]:
df['athlete'] = pd.to_numeric(df['athlete'])
athlete2019run = df[(df['datetime'].dt.year==2019) &
                    (df['activity']=='run')]['athlete'].unique()
print('Number of athletes: {} and who ran at least once in 2019: {}'
      .format(df['athlete'].unique().size, athlete2019run.size))
#df = df[~df['athlete'].isin(athlete2019run)]  # CopyWarning
df = df.drop(index=df[~df['athlete'].isin(athlete2019run)].index)
df['athlete'] = df['athlete'].astype('category')
df['athlete'] = df['athlete'].cat.remove_unused_categories()

In [ ]:
ndays = df['datetime'].dt.date.value_counts().size
print('Number of days:', ndays)
nathletes = df['athlete'].unique().size
print('Number of athletes:', nathletes)
nactivities = df['activity'].size
print('Number of activities:', nactivities)
nactivities2 = df['activity'].unique().size
print('Number of types of activities:', nactivities2)
duration = df['duration'].sum()/ndays/nathletes
print('Average duration of activity per day per athlete: {:.0f} s'.format(duration))

### Save the current dataframe as the default raw dataset

### Select 'run' as activity type

Let's analyze only running activity. In the dataset, there are two run activities: 'run' and 'virtualrun' (although the duration of 'virtual run' is insignificant).  

In [ ]:
drun = df[df['activity']=='run']['duration'].sum()/ndays/nathletes
print('Average duration of run per day per athlete: {:.0f} s'.format(drun))
dvirtualrun = df[df['activity']=='virtualrun']['duration'].sum()/ndays/nathletes
print('Average duration of virtual run per day per athlete: {:.0f} s'.format(dvirtualrun))

In [ ]:
#df = df[(df['activity']=='run') | (df['activity']=='virtualrun')]
df = df[df['activity']=='run']
df = df.drop(columns='activity')

In [ ]:
y = df[df['datetime'].dt.year == 2019][['athlete', 'duration']].groupby('athlete').sum()
norun = y[~y.any(axis='columns', skipna=False)]
print('Number of athletes who did not run in 2019:', norun.shape[0])
y = df[df['datetime'].dt.year == 2020][['athlete', 'duration']].groupby('athlete').sum()
norun = y[~y.any(axis='columns', skipna=False)]
print('Number of athletes who did not run in 2020:', norun.shape[0])

### Transform units to kilometers and minutes

For long distance running, kilometers and minutes are more convenient units for distance and duration.

In [ ]:
df['distance'] = df['distance']/1000
df['duration'] = df['duration']/60

In [ ]:
nathletes = df['athlete'].unique().size
print('Number of athletes:', nathletes)
nactivities = df.shape[0]
print('Number of run activities:', nactivities)
distance = df['distance'].sum()/ndays/nathletes
print('Average distance of run per day per athlete: {:.3g} km'.format(distance))
duration = df['duration'].sum()/ndays/nathletes
print('Average duration of run per day per athlete: {:.3g} min'.format(duration))
pace = duration / distance
print('Average pace of run per day per athlete: {:.3g} min/km'.format(pace))

Selecting only the run activity, about 4 millions of other activities the athletes performed were removed (about one fourth of the 14.5 millions activities in total).

In [ ]:
years = [2019, 2020]
for year in years:
    print('Year: {}'.format(year))
    nday = df[df['datetime'].dt.year == year]['datetime'].dt.date.value_counts().size
    print(' Number of days:', nday)
    nathlete = df[df['datetime'].dt.year == year]['athlete'].unique().size
    print(' Number of athletes:', nathlete)
    nactivity = df[df['datetime'].dt.year == year].shape[0]
    print(' Total number of activities:', nactivity)

### Update age groups to 2019

In the dataset, the age-group interval was taken from the Strava web page with the segment of the Major marathon the athlete participated. If the athlete participated in more than one Major, the age-group interval kept is from the segment with the most recent Major.  
Anyway, the age-group interval was from the year the Major occurred and this interval must be updated to 2019.  
First, we will estimate the athlete's age at her/his most recent Major as the average of the age-group interval reported. Second, the estimated age will be updated to 2019 by summing to this age the difference between 2019 and the year of the athlete's most recent Major. Finally, the updated age will be transformed to one of the existent age-group intervals.

In [ ]:
y = df[['athlete', 'age_group', 'major']
      ].drop_duplicates(subset='athlete').sort_values('athlete').reset_index(drop=True)
age_group = y['age_group'].cat.categories.to_list()
age = {'19 and under':19, '20 to 24':22.5, '25 to 34':30, '35 to 44':40,
       '45 to 54':50, '55 to 64':60, '65 to 69':67.5, '70 to 74':72.5, '75+':76}
y['age_group'] = y['age_group'].map(age).astype(float)
f = lambda x: np.array(x, dtype=int).max()
y['age_group'] = y['age_group'] + (2019 - y['major'].str.findall(r'[0-9]+').apply(f))
bins = np.r_[18, np.array([a.split(' to ')[0] for a in age_group[1:-1]],
                          dtype=int), 75, 100]
y['age_group'] = pd.cut(y['age_group'], bins=bins, right=False, labels=age_group)
df = df.drop('age_group', axis=1).join(y.drop('major', axis=1).set_index('athlete'),
                                       on='athlete')
df = df[['datetime', 'athlete', 'gender', 'age_group',
         'distance', 'duration', 'country', 'major']]

### Merge the 9 age groups into 3 groups

To simplify the data analysis per age group, let's reduce from nine to three age groups.

In [ ]:
df.drop_duplicates(subset='athlete')[['athlete', 'age_group', 'gender']
                                    ].groupby(['age_group',
                                               'gender']).count().unstack(level=0)

In [ ]:
age_groups = {'19 and under':'18 - 34', '20 to 24':'18 - 34', '25 to 34':'18 - 34',
              '35 to 44':'35 - 54', '45 to 54':'35 - 54',
              '55 to 64':'55 +', '65 to 69':'55 +', '70 to 74':'55 +', '75+':'55 +'}
df['age_group'] = df['age_group'].map(age_groups).astype('category')
df.drop_duplicates(subset='athlete')[['athlete', 'age_group', 'gender']
                                    ].groupby(['age_group',
                                               'gender']).count().unstack(level=0)

### Correct some country names 

See https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes

In [ ]:
df['country'].replace(
    {'The Netherlands': 'Netherlands', 'Suomi Finland': 'Finland',
     'Moldavia': 'Moldova', 'Republic of Korea': 'South Korea',
     'The United Arab Emirates': 'United Arab Emirates',
     'Luzon': 'Philippines', 'Hellas': 'Greece', 'Cyprus - Kibris': 'Cyprus',
     'Mongolian': 'Mongolia', 'The Savior': 'El Salvador',
     'Balgariya': 'Bulgaria', 'the two seas': 'Bahrain',
     'Føroyar': 'Faroe Islands', 'Saudi': 'Saudi Arabia',
     'Amman': 'Jordan', 'The Bahamas': 'Bahamas', "Timor Lorosa'e": 'East Timor',
     'Diameter': np.nan}, inplace=True)
df['country'] = df['country'].astype('category')

## Export dataset

See a comparison on [formats to save Pandas data](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d).

In [ ]:
df.sort_values(['datetime', 'athlete'], inplace=True)
df.reset_index(inplace=True, drop=True)
df

In [ ]:
df.info(memory_usage='deep')

### Full dataset with running activities

In [ ]:
df.to_parquet(os.path.join(path2, 'run_ww_2019_2020.parquet'), engine='pyarrow')

### Test file

In [ ]:
df = pd.read_parquet(os.path.join(path2, 'run_ww_2019_2020.parquet'))
#df['athlete'] = df['athlete'].astype('category')  # bug in parquet
df

In [ ]:
df.info(memory_usage='deep')